<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">ESO Science Archive</h1> 
  <h2 style="color: #0281c9; font-weight: bold;">Jupyter Notebooks</h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

# **Query using TAP**

The Table Access Protocol (TAP) allows you to perform advanced searches on ESO's Science Archive using the Astronomical Data Query Language (ADQL).

You can query different tables depending on your science goal:

- **`ivoa.ObsCore`** — standardized metadata for fully calibrated Phase 3 science-ready products
- **`dbo.raw`** — metadata for raw observational data (spanning all ESO instruments)
- **`ist.<instrument_name>`** — instrument-specific metadata tables (e.g., `ist.midi`, `ist.muse`), offering fine-grained control

<hr style="border:2px solid #0281c9"> </hr>

# **Importing and basic usage of astroquery.eso**

In [1]:
import astroquery # import astroquery
print(f"astroquery version: {astroquery.__version__}") # check the version of astroquery

astroquery version: 0.4.11.dev10245


In [2]:
from astroquery.eso import Eso # import the ESO module from astroquery

In [3]:
eso = Eso() # create an instance of the ESO class 

# **Query the ESO Archive using the TAP Service**

## **Reduced data search (ivoa.ObsCore)**

This example shows how to query the **VEGAS** survey (VST Early-type GAlaxieS Survey) for **science-ready, reduced imaging** of bright early-type galaxies from the NGC catalog. These data products are part of ESO's **Phase 3 collection** and include calibrated FITS images.

We filter the query for:
- `data_collection = 'VEGAS'`
- Target names from the NGC catalog
- Spatial resolution constraints

Such products are ideal for studying galaxy morphology and low-surface-brightness features.

First it is always useful to check which columns are available in the table you want to query. This can be done using ``eso.query_surveys(help=True)``, which queries `ivoa.ObsCore`.

In [18]:
eso.query_surveys(help=True)

INFO: 
Columns present in the table ivoa.ObsCore:
    column_name     datatype    xtype     unit 
------------------- -------- ----------- ------
           abmaglim   double                mag
     access_estsize     long              kbyte
      access_format     char                   
         access_url     char                   
      bib_reference     char                   
        calib_level      int                   
dataproduct_subtype     char                   
   dataproduct_type     char                   
              dp_id     char                   
             em_max   double                  m
             em_min   double                  m
       em_res_power   double                   
             em_xel     long                   
      facility_name     char                   
             filter     char                   
            gal_lat   double                deg
            gal_lon   double                deg
    instrument_name     char          

In [19]:
adql_query = """
            SELECT TOP 30 target_name, s_ra, s_dec, instrument_name, 
                obs_collection, dataproduct_type, s_pixel_scale, 
                calib_level, t_min, proposal_id, dp_id
            FROM ivoa.ObsCore
            WHERE obs_collection = 'vegas'
                AND calib_level = 3
                AND dataproduct_type = 'image'
                AND s_pixel_scale < 0.4
                AND target_name LIKE 'NGC%'
            ORDER BY t_min DESC
            """

table = eso.query_tap_service(adql_query)
table["target_name", "s_ra", "s_dec", "instrument_name", "dataproduct_type", "s_pixel_scale", "dp_id"][:5]

target_name,s_ra,s_dec,instrument_name,dataproduct_type,s_pixel_scale,dp_id
,deg,deg,,,arcsec,
object,float64,float64,object,object,float64,object
ngc1533,62.389416,-56.114055,OMEGACAM,image,0.21,ADP.2021-02-01T14:04:33.669
ngc1533,62.444166,-56.100777,OMEGACAM,image,0.21,ADP.2021-02-01T14:04:33.667
ngc5018,197.876458,-19.336277,OMEGACAM,image,0.2099,ADP.2021-02-01T14:04:33.675
ngc3379,161.513666,12.642055,OMEGACAM,image,0.2099,ADP.2021-02-01T14:04:33.679
ngc5018,197.876458,-19.336277,OMEGACAM,image,0.2099,ADP.2021-02-01T14:04:33.673


## **Raw data search (dbo.raw)**

This query targets **raw narrow-band imaging** around Hα (656.3 nm) for the galaxy **NGC 3627**.

Key filters applied:
- Cone search around galaxy coordinates (RA/Dec) with 0.1° radius
- Spectral band includes Hα
- Filter width < 20 nm to exclude broadband observations
- Only science exposures (`dp_cat = 'SCIENCE'`)

These raw frames are useful for custom data reduction workflows focused on ionized gas and star formation in nearby galaxies.

In [14]:
from astropy.coordinates import SkyCoord # import the SkyCoord class from the astropy.coordinates module

target = "NGC 3627" # set the target to NGC 3627
coords = SkyCoord.from_name(target) # create a SkyCoord object from the name of the source

Again, before running the query, it is useful to check which columns are available in the table you want to query. This can be done using ``eso.query_main(help=True)``, which queries `dbo.raw`.

In [20]:
eso.query_main(help=True)

INFO: 
Columns present in the table dbo.raw:
    column_name     datatype    xtype     unit 
------------------- -------- ----------- ------
     access_estsize     long              kbyte
         access_url     char                   
       datalink_url     char                   
           date_obs     char                   
                dec   double                deg
            dec_pnt   double                deg
        det_chip1id     char                   
   det_chop_ncycles    short                   
            det_dit    float                  s
          det_expid    short                   
           det_ndit    short                   
             dp_cat     char                   
              dp_id     char                   
            dp_tech     char                   
            dp_type     char                   
            ecl_lat   double                deg
            ecl_lon   double                deg
          exp_start     char   timestamp   

In [38]:
adql_query = f"""
            SELECT *
            FROM dbo.raw
            WHERE dp_cat = 'SCIENCE'
                AND 1 = INTERSECTS(s_region, CIRCLE('', {coords.ra.value},  {coords.dec.value}, 0.1))  -- defining a cone of 0.1 deg radius around NGC 3627
                AND lambda_min < 656 
                AND lambda_max > 656                                -- Ensure Hα falls in the filter bandpass
                AND lambda_max - lambda_min < 20                    -- Narrow-band filter (~< 20 nm)
            ORDER BY mjd_obs
            """ 

table = eso.query_tap_service(adql_query)
table["object", "ra", "dec", "instrument", "exposure", "tel_ambi_fwhm_start", "date_obs", "prog_id", "dp_id", "access_url"][:5]

object,ra,dec,instrument,exposure,tel_ambi_fwhm_start,date_obs,prog_id,dp_id,access_url
,deg,deg,,s,arcsec,,,,
object,float64,float64,object,float32,float32,object,object,object,object
NGC-3627,170.06142694,12.99157,WFI,89.917,0.68,2019-02-08T07:03:09.793,0102.A-9002(A),WFI.2019-02-08T07:03:09.793,https://dataportal.eso.org/dataPortal/file/WFI.2019-02-08T07:03:09.793
NGC-3627,169.97285888,12.95482,WFI,89.917,0.68,2019-02-08T07:12:44.346,0102.A-9002(A),WFI.2019-02-08T07:12:44.347,https://dataportal.eso.org/dataPortal/file/WFI.2019-02-08T07:12:44.347
NGC-3627,169.98158694,12.96159,WFI,89.917,0.68,2019-02-08T07:15:41.075,0102.A-9002(A),WFI.2019-02-08T07:15:41.076,https://dataportal.eso.org/dataPortal/file/WFI.2019-02-08T07:15:41.076
NGC-3627,169.97238111,12.955,WFI,359.917,0.68,2019-02-08T07:18:33.565,0102.A-9002(A),WFI.2019-02-08T07:18:33.565,https://dataportal.eso.org/dataPortal/file/WFI.2019-02-08T07:18:33.565
NGC-3627,169.98128888,12.96175,WFI,359.917,0.68,2019-02-08T07:25:59.174,0102.A-9002(A),WFI.2019-02-08T07:25:59.174,https://dataportal.eso.org/dataPortal/file/WFI.2019-02-08T07:25:59.174


## **Raw (instrument specific) data search (ist.[instrument])**

As with ``eso.query_main`` vs ``eso.query_instrument`` we can also query the instrument specific tables directly in tap using the `ist.[instrument]` syntax.

For example in this case we can, for example, query the `ist.muse` table directly to get observations from **MUSE** (IFU) only, but with more instrument specific columns available.

As always, its good to check which columns are available in the table you want to query. This can be done using ``eso.query_instrument("MUSE", help=True)``, which queries `ist.muse`.

In [30]:
eso.query_instrument("MUSE", help=True)

INFO: 
Columns present in the table ist.MUSE:
      column_name        datatype    xtype       unit  
------------------------ -------- ----------- ---------
          access_estsize     long                 kbyte
              access_url     char                      
            ag_fwhmx_avg    float                arcsec
            ag_fwhmy_avg    float                arcsec
            datalink_url     char                      
                date_obs     char                      
                     dec   double                   deg
                det_binx      int                      
                det_biny      int                      
                det_ndit      int                      
        det_read_curname     char                      
           dimm_fwhm_avg    float                arcsec
           dimm_fwhm_rms    float                arcsec
                  dp_cat     char                      
                   dp_id     char                      
  

In [ ]:
adql_query = f"""
            SELECT *
            FROM ist.muse
            WHERE dp_cat = 'SCIENCE'
                AND 1 = INTERSECTS(s_region, CIRCLE('', {coords.ra.value},  {coords.dec.value}, 0.1))
            ORDER BY mjd_obs
            """ 

table = eso.query_tap_service(adql_query)
table["object", "ra", "dec", "instrument", "exposure", "tel_ambi_fwhm_start", "date_obs", "prog_id", "dp_id", "access_url"][:5]

object,ra,dec,instrument,exposure,tel_ambi_fwhm_start,date_obs,prog_id,dp_id,access_url
,deg,deg,,s,arcsec,,,,
object,float64,float64,object,float32,float32,object,object,object,object
NGC3627,170.072929,12.98949,MUSE,645.0,0.74,2018-01-25T07:19:50.000,1100.B-0651(A),MUSE.2018-01-25T07:19:50.461,https://dataportal.eso.org/dataPortal/file/MUSE.2018-01-25T07:19:50.461
NGC3627,170.072986,12.98949,MUSE,645.0,0.64,2018-01-25T07:36:07.000,1100.B-0651(A),MUSE.2018-01-25T07:36:07.480,https://dataportal.eso.org/dataPortal/file/MUSE.2018-01-25T07:36:07.480
NGC3627,170.072986,12.98955,MUSE,645.0,0.56,2018-01-25T07:48:51.000,1100.B-0651(A),MUSE.2018-01-25T07:48:51.468,https://dataportal.eso.org/dataPortal/file/MUSE.2018-01-25T07:48:51.468
NGC3627,170.072929,12.98955,MUSE,645.0,0.51,2018-01-25T08:05:08.000,1100.B-0651(A),MUSE.2018-01-25T08:05:08.462,https://dataportal.eso.org/dataPortal/file/MUSE.2018-01-25T08:05:08.462
NGC3627,170.056157,12.98989,MUSE,645.0,1.39,2018-02-19T05:54:22.000,1100.B-0651(A),MUSE.2018-02-19T05:54:22.559,https://dataportal.eso.org/dataPortal/file/MUSE.2018-02-19T05:54:22.559


**Note:** For more information about the TAP and how to write ADQL queries, refer to the following resources:

* ESO TAP+ documentation: Describes ESO’s implementation of TAP and the available services.
* IVOA TAP standard: The official specification from the International Virtual Observatory Alliance.
* ADQL specification: Defines the query language used to interact with TAP service

<hr style="border:2px solid #0281c9"> </hr>